# Clusterização: K-Means

Para mais informações de como rodar, veja o arquivo `orientacao.md`.

&ensp;O K-Means é um algoritimo não-supervisionado de clusterização, que tem como objetivo de agrupar dados semelhantes em grupos que tenham características específicas e similaridades entre si. Logo, sua utilização no projeto será voltada para a criação de diferentes grupos de usuários, com base nos dados presentes na base com sua utilização do plano de saúde.

- Importando das bibliotecas necessárias:

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import seaborn as sns


- Leitura e informações do Data Frame (esse dataframe estará disponível após a execução do notebook `preprocessamento.ipynb`):

In [ ]:
df = pd.read_csv('./data_updated.csv')
df.info()

In [ ]:
df.describe()

&emsp;O _Elbow Method_ é uma forma de encontrar a quantidade ideal de clusters (quanto vale K) para se utilizar no algoritimo K-Means, uma vez que o algoritimo precisa que a quantidade de clusters que serão designados seja definido previamente.

In [ ]:

numeric_columns = ["faixa-etaria_encoded","valor-pago-sinistro_standardized", "descricao_servico_sinistro_encoded", "doenca_relacionadas_encoded", "sexo_encoded", "tipo_servico_encoded"]

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df[numeric_columns])
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.grid()
plt.show()

&emsp; O código realiza a criação de um modelo K-Means para agrupar os dados numéricos em 4 clusters, sendo que a quantidade de clusters foi definida anteriormente a partir do Elbow Method, uma técnica que auxilia na escolha do valor ideal de K. Após o treinamento do modelo, cada ponto é classificado em um dos 4 clusters, e os centros de cada cluster são exibidos. Além disso, a quantidade de elementos pertencentes a cada cluster é calculada e apresentada no final.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0, max_iter=100000)
numeric_cols = df[numeric_columns]

X = numeric_cols
kmeans.fit(X)

df['cluster'] = kmeans.predict(X)
print(kmeans.cluster_centers_)
df["cluster"].value_counts()

&emsp; O código aplica PCA, uma técnica de redução dimensional usada para simplificar grandes conjuntos de dados, preservando o máximo de variabilidade possível. O PCA transforma os dados originais em novas variáveis chamadas componentes principais, que são combinações lineares das variáveis originais.

&emsp; Dessa forma, os dados são reduzidos a 2 componentes principais para facilitar a visualização dos clusters formados pelo K-Means. Esses componentes são adicionados ao DataFrame e usados para criar um gráfico de dispersão, onde os pontos são coloridos de acordo com o cluster. Os centróides dos clusters também são plotados no gráfico para mostrar os centros dos grupos como pode ser obeservado no gráfico.

In [ ]:
from sklearn.decomposition import PCA

# Aplicar PCA para reduzir os dados a 2 componentes principais
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Adicionar os componentes principais ao DataFrame para visualização
df['PCA1'] = X_pca[:, 0]
df['PCA2'] = X_pca[:, 1]

# Plotar os resultados em um gráfico de dispersão
plt.figure(figsize=(20, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='cluster', data=df, palette='viridis', style='cluster', markers=['o', 's', '^'])

# Plotar os centróides dos clusters em cima do gráfico PCA
centroids_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=300, c='red', marker='X', label='Centroides')

plt.title('Visualização dos Clusters com PCA e KMeans')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()


&emsp; Este código verifica se algumas colunas estão presentes no DataFrame df. Se estiverem, ele gera um gráfico de dispersão (scatter plot) onde os dados são coloridos de acordo com os clusters definidos pelo K-Means. O gráfico exibe a relação entre a quantidade e o valor pago do sinistro, facilitando a visualização dos agrupamentos nos dados. Caso as colunas não estejam presentes, é exibida uma mensagem informando que as colunas necessárias estão ausentes.

In [ ]:

# Verificar se as colunas existem no DataFrame
if 'quantidade_standardized' in df.columns and 'valor-pago-sinistro_standardized' in df.columns and 'cluster' in df.columns:
    plt.figure(figsize=(10, 6))

    # Scatter plot com coloração pelos clusters
    sns.scatterplot(
        x="quantidade_standardized",
        y="valor-pago-sinistro_standardized",
        hue='cluster',
        data=df,
        palette='viridis',
        s=100
    )

    plt.title('Clusters de KMeans')
    plt.xlabel('Quantidade')
    plt.ylabel('Valor Pago Sinistro')
    plt.legend()
    plt.show()
else:
    print("As colunas necessárias não estão presentes no DataFrame.")

&emsp; Este código cria uma visualização 3D dos clusters formados pelo K-Means usando Plotly Express. Ele começa criando um DataFrame df_clusters com as colunas padronizadas e a coluna de clusters. Em seguida, gera um gráfico de dispersão com três dimensões onde as coordenadas são baseadas em três variáveis padronizadas: 'faixa-etaria_encoded_standardized', 'valor-pago-sinistro_standardized', e 'descricao_servico_sinistro_encoded_standardized'.

In [ ]:
import plotly.express as px
import pandas as pd

df_clusters = pd.DataFrame({
    'faixa-etaria_encoded': X['faixa-etaria_encoded'],
    'valor-pago-sinistro_standardized': X['valor-pago-sinistro_standardized'],
    'descricao_servico_sinistro_encoded': X['descricao_servico_sinistro_encoded'],
    'doenca_relacionadas_encoded': X['doenca_relacionadas_encoded'],
    'sexo_encoded': X['sexo_encoded'],
    'tipo_servico_encoded': X['tipo_servico_encoded'],
    'cluster': df['cluster']
})

fig = px.scatter_3d(
    df_clusters,
    x='faixa-etaria_encoded',
    y='valor-pago-sinistro_standardized',
    z='descricao_servico_sinistro_encoded',
    color='cluster',
    title='Clusters de KMeans em 3D',
    opacity=1.0,
    size_max=10
)

fig.show()

&emsp; Este código cria uma matriz de dispersão (scatter matrix) usando Plotly Express para visualizar a relação entre várias variáveis dos clusters. Cada ponto é colorido de acordo com o cluster a que pertence, facilitando a comparação das variáveis dentro de cada cluster.



In [ ]:
fig_matrix = px.scatter_matrix(
    df_clusters,
    dimensions=['descricao_servico_sinistro_encoded', 'faixa-etaria_encoded', 'valor-pago-sinistro_standardized', 'tipo_servico_encoded', 'sexo_encoded', 'doenca_relacionadas_encoded'],
    color='cluster',
    title='Scatter Matrix entre Variáveis por Cluster',
    labels={   # Definir rótulos claros para as variáveis
        'descricao_servico_sinistro_encoded': 'Código Serviço',
        'faixa-etaria_encoded': 'Faixa Etária',
        'valor-pago-sinistro_standardized': 'Valor Pago',
        'tipo_servico_encoded': 'Tipo Serviço',
        'sexo_encoded': 'Sexo',
        'doenca_relacionadas_encoded': 'Doenças Relacionadas'
    },
    height=900,  # Aumentar a altura para reduzir sobreposição
    width=900    # Aumentar a largura
)

# Ajustando o layout para evitar sobreposição das labels
fig_matrix.update_layout(
    title={'x':0.5},  # Centraliza o título
    font=dict(size=10),  # Reduz o tamanho da fonte
    margin=dict(l=50, r=50, b=100, t=100),  # Ajusta as margens
    autosize=False
)

# Mostrar o gráfico
fig_matrix.show()


- Exportando o cluster para um arquivo CSV:

In [28]:
import pickle
df.to_csv('./data_updated_clustered_kmeans.csv', index=False)
pickle.dump(kmeans, open('./assets/kmeans_model.pkl', 'wb'))